In [ ]:
import sys
import os
from matplotlib.colors import from_levels_and_colors
import scipy.io as sio
sys.path.append(os.environ['GOTMWORK_ROOT']+'/tools', )
from gotmanalysis import *
np.seterr(all='raise')
%matplotlib inline

In [ ]:
casename = 'JRA55-do_Global_dampV5d'
forcing_reg_type = 'BG12'
tmname = 'KPP-CVMix'
update_data = False
plot_figure = True

In [ ]:
# check forcing_reg_type
fr_list = ['BG12', 'LF17']
if forcing_reg_type not in fr_list:
    print('Forcing regime {} not supported. Stop.'.format(forcing_reg_type))

In [ ]:
# lists
timetag_list = ['20090101-20090131',
                '20090201-20090228',
                '20090301-20090331',
                '20090401-20090430',
                '20090501-20090531',
                '20080601-20080630',
                '20080701-20080731',
                '20080801-20080831',
                '20080901-20080930',
                '20081001-20081031',
                '20081101-20081130',
                '20081201-20081231']
ntag = len(timetag_list)

In [ ]:
# list of parameters

# McWilliams et al., 1997
c_msm97 = 'navy'
msm97_la = np.array([0.3, 1000])
msm97_ustar = np.ones(msm97_la.size)*6.1e-3
msm97_B0 = np.ones(msm97_la.size)*5/4200/1000*2e-4*9.8
msm97_mld = np.ones(msm97_la.size)*35
msm97_hLL = msm97_la**2*msm97_B0*msm97_mld/msm97_ustar**3

# Li et al., 2005
# h/LL = Ho * \beta * H / 4
c_l05 = 'skyblue'
l05_la_range_s1 = [0.2, 10]
l05_hLL_s1 = 0.0
l05_la_s2 = 0.34
l05_hLL_range_s2 = [0, 100]
l05_la_s3 = 0.34
l05_hLL_s3 = 0.25*8.4/4 # (surface heating)

# Harcourt and D'Asaro 2008
c_hd08 = 'orange'
hd08_la_s1 = np.array([0.331, 0.341, 0.351, 0.360, 0.366, 0.375, 0.381, 0.387,
                       0.306, 0.316, 0.325, 0.332, 0.338, 0.346, 0.351, 0.357,
                       0.291, 0.301, 0.309, 0.316, 0.322, 0.329, 0.333, 0.339,
                       0.270, 0.278, 0.286, 0.292, 0.298, 0.304, 0.308, 0.313])
hd08_la_s2 = np.array([0.275, 0.288, 0.300, 0.310, 0.320, 0.331, 0.339, 0.347])
hd08_la_s3 = np.array([0.404, 0.417, 0.435, 0.364, 0.364, 0.364, 0.364])
hd08_la_s4 = np.array([0.270, 0.278, 0.286, 0.292, 0.298, 0.304, 0.308, 0.313])
hd08_hLL_s1 = np.ones(hd08_la_s1.size)*1e-8
hd08_hLL_s2 = np.ones(hd08_la_s2.size)*1e-8
hd08_hLL_s3 = np.ones(hd08_la_s3.size)*1e-8
hd08_hLL_s4 = np.ones(hd08_la_s4.size)*1e-8

# Grant and Belcher, 2009
c_gb09 = 'royalblue'
gb09_la = np.array([0.15, 0.2, 0.3, 0.4, 0.8, 1.5])
gb09_hLL = np.ones(gb09_la.size)*1e-8

# Van Roekel et al., 2012
# vr12_mflag = 
c_vr12 = 'orchid'
vr12_la = np.array([0.18, 0.23, 0.29, 0.37, 0.49])
vr12_us0 = np.array([0.022, 0.038, 0.063, 0.100, 0.162])
vr12_ustar = np.ones(vr12_la.size)*5.3e-3
vr12_mld = np.ones(vr12_la.size)*35
vr12_B0 = np.ones(vr12_la.size)*5/4200/1000*2e-4*9.8
vr12_hLL = vr12_la**2*vr12_B0*vr12_mld/vr12_ustar**3

# Pearson et al., 2015 (surface heating)
c_p15 = 'firebrick'
p15_LL_s1 = np.array([62, 93, 124, 186, 248, 372, 496])
p15_hi_s1 = 53
p15_hiLL_s1 = p15_hi_s1/p15_LL_s1
p15_hLL_s1 = p15_hiLL_s1/(1+3.5*p15_hiLL_s1)
p15_la_s1 = np.ones(p15_LL_s1.size)*0.3
p15_LL_s2 = np.array([93, 124, 186, 248, 372])
p15_hi_s2 = 36
p15_hiLL_s2 = p15_hi_s2/p15_LL_s2
p15_hLL_s2 = p15_hiLL_s2/(1+3.5*p15_hiLL_s2)
p15_la_s2 = np.ones(p15_LL_s2.size)*0.3

# Reichl et al., 2016
c_r16 = 'mediumpurple'
r16_B0 = np.ones(vr12_la.size)*5/4200/1000*2e-4*9.8
r16_la_range = [0.25, 0.7]
r16_hLL = 0.0

# Li and Fox-Kemper, 2017
c_lf17 = 'pink'
lf17_data = sio.loadmat('LF17_La_hLL.mat')
lf17_la_tmp = lf17_data['la']
lf17_la_tmp = np.where(np.isinf(lf17_la_tmp), np.nan, lf17_la_tmp)
lf17_la = lf17_la_tmp[~np.isnan(lf17_la_tmp)]
lf17_hLL_tmp = lf17_data['hL']
lf17_hLL_tmp = np.where(np.isinf(lf17_hLL_tmp), np.nan, lf17_hLL_tmp)
lf17_hLL = lf17_hLL_tmp[~np.isnan(lf17_la_tmp)]

In [ ]:
# paths
fig_root = os.environ['GOTMFIG_ROOT']+'/'+casename

In [ ]:
# get data
mon_lat = []
mon_lon = []
mon_time = []
mon_laturb = []
mon_bflux = []
mon_ustar = []
mon_hbl = []
for j in np.arange(ntag):
    timetag = timetag_list[j]
    # read surface forcing  data
    s1data_root = os.environ['GOTMRUN_ROOT']+'/'+casename+'/VR1m_DT600s_'+timetag
    s2data_root = os.environ['GOTMFIG_ROOT']+'/data/'+casename+'/VR1m_DT600s_'+timetag
    os.makedirs(s2data_root, exist_ok=True)
    os.makedirs(fig_root, exist_ok=True)
    basepath = s1data_root+'/'+tmname
    s2data_name = s2data_root+'/data_mapts_surface_forcing_'+tmname+'.npz'
    if update_data or not os.path.isfile(s2data_name):
        # update data
        print('Updating data for {} ...'.format(timetag))
        loclist = sorted(os.listdir(basepath))
        pathlist = [basepath+'/'+x+'/gotm_out_s1.nc' for x in loclist]
        godmobj = GOTMOutputDataMap(pathlist)
        laturb = np.zeros([godmobj.ncase, godmobj.ntime-1])
        bflux = np.zeros([godmobj.ncase, godmobj.ntime-1])
        ustar = np.zeros([godmobj.ncase, godmobj.ntime-1])
        hbl = np.zeros([godmobj.ncase, godmobj.ntime-1])
        lat = godmobj.lat
        lon = godmobj.lon
        time = godmobj.time[1:]
        for i in np.arange(godmobj.ncase):
            if np.mod(i, 100) == 0:
                print('{:6.2f} %'.format(i/godmobj.ncase*100.0))
            tmp = GOTMOutputData(godmobj._paths[i], init_time_location=False)
            laturb[i,:] = tmp.read_timeseries('La_Turb', ignore_time=True).data[1:]
            bflux[i,:] = tmp.read_timeseries('bflux', ignore_time=True).data[1:]
            ustar[i,:] = tmp.read_timeseries('u_taus', ignore_time=True).data[1:]
            hbl[i,:] = tmp.read_timeseries('mld_deltaR', ignore_time=True).data[1:]
        # save data
        np.savez(s2data_name, laturb=laturb, bflux=bflux, ustar=ustar, hbl=hbl,
                 lon=lon, lat=lat, time=time)
    else:
        # read data
        tmp = np.load(s2data_name)
        lat = tmp['lat']
        lon = tmp['lon']
        time = tmp['time']
        laturb = tmp['laturb']
        bflux = tmp['bflux']
        ustar = tmp['ustar']
        hbl = tmp['hbl']
    # append to monthly lists
    mon_lat.append(lat)
    mon_lon.append(lon)
    mon_time.append(time)
    mon_laturb.append(laturb)
    mon_bflux.append(bflux)
    mon_ustar.append(ustar)
    mon_hbl.append(hbl)
    

In [ ]:
# turbulent Langmuir number
fl_laturb = np.concatenate([mon_laturb[i].flatten() for i in np.arange(ntag)])
# surface buoyancy flux
fl_bflux = np.concatenate([mon_bflux[i].flatten() for i in np.arange(ntag)])
# friction velocity
fl_ustar = np.concatenate([mon_ustar[i].flatten() for i in np.arange(ntag)])
# boundary layer depth
fl_hbl = np.concatenate([mon_hbl[i].flatten() for i in np.arange(ntag)])
# remove data points where friction velocity is zero
inds = fl_ustar==0
print('Invalid data points: {:6.2f}%'.format(np.sum(inds)/fl_ustar.size*100))
fl_laturb[inds] = np.nan
fl_bflux[inds] = np.nan
fl_ustar[inds] = np.nan
fl_hbl[inds] = np.nan
fl_laturb = fl_laturb[~np.isnan(fl_laturb)]
fl_bflux = fl_bflux[~np.isnan(fl_bflux)]
fl_ustar = fl_ustar[~np.isnan(fl_ustar)]
fl_hbl = fl_hbl[~np.isnan(fl_hbl)]
# h_b/L_L
fl_hLL = -fl_bflux*fl_hbl/(fl_ustar**3)*fl_laturb**2

# --------

# Surface cooling
xdata = np.copy(fl_laturb)
ydata = np.copy(fl_hLL)
# remove data points where surface buoyancy flux is positive (stable) 
print(ydata)
inds = ydata < 0
nneg = np.sum(inds) 
print('Negative h_b/L_L: {:6.2f}%'.format(nneg/ydata.size*100))
xdata[inds] = np.nan
ydata[inds] = np.nan
xdata = xdata[~np.isnan(xdata)]
ydata = ydata[~np.isnan(ydata)]

# get the bi-dimensional histogram in log-log space
xdata = np.log10(xdata)
ydata = np.log10(ydata)
# range of power
xpr = [-1, 1]
ypr = [-3, 3]
# range
xlims = [10**i for i in xpr]
ylims = [10**i for i in ypr]
hist, xi, yi, c = plt.hist2d(xdata, ydata, range=(xpr,ypr), bins=100)
# clean the figure
plt.clf()
# get the centers from the edges
xi = 0.5*(xi[0:-1]+xi[1:])
yi = 0.5*(yi[0:-1]+yi[1:])
# convert back to actual values
xi = 10**xi
yi = 10**yi

# --------

# Surface heating
xdata = np.copy(fl_laturb)
ydata = np.copy(fl_hLL)
# remove data points where surface buoyancy flux is negative (unstable) 
print(ydata)
inds = ydata > 0
npos = np.sum(inds) 
print('positive h_b/L_L: {:6.2f}%'.format(npos/ydata.size*100))
xdata[inds] = np.nan
ydata[inds] = np.nan
xdata = xdata[~np.isnan(xdata)]
ydata = ydata[~np.isnan(ydata)]

# get the bi-dimensional histogram in log-log space
xdata = np.log10(xdata)
ydata = np.log10(-ydata)
# range of power
xpr = [-1, 1]
ypr = [-3, 3]
# range
xlims = [10**i for i in xpr]
ylims = [10**i for i in ypr]
hist2, xi2, yi2, c2 = plt.hist2d(xdata, ydata, range=(xpr,ypr), bins=100)
# clean the figure
plt.clf()
# get the centers from the edges
xi2 = 0.5*(xi2[0:-1]+xi2[1:])
yi2 = 0.5*(yi2[0:-1]+yi2[1:])
# convert back to actual values
xi2 = 10**xi2
yi2 = 10**yi2

In [ ]:
def plot_dist_4p(hst, xi, yi, axis=None, filled=False, **kwargs):
    """Plot bi-dimensional histogram. Show the contours of the
       histogram which enclose the highest 30%, 60%, 90% and 99%
       centered distribution.
       
    :his: (2D numpy array) bi-dimensional histogram
    :xi: (1D numpy array) centers of x dimension
    :yi: (1D numpy array) centers of y dimension
    :axis: (matplotlib.axes, optional) axis to plot figure on
    :filled: (bool) filled contour if True
    :return: (matplotlib figure object) figure
    
    """
    # use curret axis if not specified
    if axis is None:
        axis = plt.gca()
    hsum = np.sum(hst)
    hlist = -np.sort(-hst.flatten())/hsum
    hcum = np.cumsum(hlist)
    vl = [0.3, 0.6, 0.9, 0.99]
    nv = len(vl)
    vlev = np.zeros(nv)
    for i in np.arange(nv):
        ind = np.argmin(abs(hcum-vl[i]))
        vlev[i] = hlist[ind]
    pdfData = hst/hsum
    pdfData[pdfData==0] = 1e-12
    if not filled:
        fig = axis.contour(xi, yi, np.log10(np.transpose(pdfData)), levels=np.log10(vlev[::-1]), **kwargs)
    else:
        fig = axis.contourf(xi, yi, np.log10(np.transpose(pdfData)), levels=np.log10(vlev[::-1]), **kwargs)
    return fig

In [ ]:
def plot_regime_diagram_BG12_background(axis=None):
    """Plot the background of the regime diagram 
       following Fig. 3 of Belcher et al., 2012
    
    """
    if axis is None:
        axis = plt.gca()
        
    # range of power
    xpr = [-1, 1]
    ypr = [-3, 3]
    # range
    xlims = [10**i for i in xpr]
    ylims = [10**i for i in ypr]
    # size of x and y
    nx = 500
    ny = 500
    xx = np.logspace(xpr[0], xpr[1], nx)
    yy = np.logspace(ypr[0], ypr[1], ny)
    zz1 = np.zeros([nx, ny])
    zz2 = np.zeros([nx, ny])
    zz3 = np.zeros([nx, ny])
    for i in np.arange(nx):
        for j in np.arange(ny):
            zz1[i,j] = 2*(1-np.exp(-0.5*xx[i]))
            zz2[i,j] = 0.22*xx[i]**(-2)
            zz3[i,j] = 0.3*xx[i]**(-2)*yy[j]
    zz = zz1 + zz2 + zz3
    axis.contourf(xx, yy, np.transpose(np.log10(zz)),
                  levels=[-0.1, 0, 0.1, 0.25, 0.5, 1, 2, 3],
                  cmap='summer', extend='both')
    axis.contour(xx, yy, np.transpose(np.log10(zz)),
                  levels=[-0.1, 0, 0.1, 0.25, 0.5, 1, 2, 3],
                  colors='darkgray')
    axis.contour(xx, yy, np.transpose(zz1/zz), levels=0.9, colors='k',
                linestyles='-', linewidths=2)
    axis.contour(xx, yy, np.transpose(zz2/zz), levels=0.9, colors='k',
                linestyles='-', linewidths=2)
    axis.contour(xx, yy, np.transpose(zz3/zz), levels=0.9, colors='k',
                linestyles='-', linewidths=2)
    axis.set_xlim(xlims)
    axis.set_ylim(ylims)
    axis.set_xscale('log')
    axis.set_yscale('log')
    axis.set_xlabel('La$_t$')
    axis.set_ylabel('$h/L_L$')
    axis.set_aspect(aspect=1/3)
    axis.text(0.11, 4e-3, 'Langmuir', bbox=dict(boxstyle="square",ec='k',fc='w'))
    axis.text(3, 4e-3, 'Wind', bbox=dict(boxstyle="square",ec='k',fc='w'))
    axis.text(0.13, 1e2, 'Convection', bbox=dict(boxstyle="square",ec='k',fc='w'))

In [ ]:
def plot_regime_diagram_BG12(hist, xi, yi, axis=None):
    """Plot regime diagram following Belcher et al., 2012
    
    """
    # background following Fig. 3 of Belcher et al., 2012
    plot_regime_diagram_BG12_background(axis=axis)
    
    # plot bi-dimensional histogram
    plot_dist_4p(hist, xi, yi, axis=axis, colors='white', linestyles='-')


In [ ]:
def plot_regime_diagram_heating(hist, xi, yi, axis=None):
    if axis is None:
        axis = plt.gca()
        
    # plot bi-dimensional histogram
    plot_dist_4p(hist, xi, yi, axis=axis, colors='black', linestyles='-')
    # range of power
    xpr = [-1, 1]
    ypr = [-3, 3]
    # range
    xlims = [10**i for i in xpr]
    ylims = [10**i for i in ypr]
    axis.set_xlim(xlims)
    axis.set_ylim(ylims)
    axis.set_xscale('log')
    axis.set_yscale('log')
    axis.set_xlabel('La$_t$')
    axis.set_ylabel('$-h/L_L$')
    axis.set_aspect(aspect=1/3)
    axis.text(0.11, 4e-3, 'Langmuir', bbox=dict(boxstyle="square",ec='k',fc='w'))
    axis.text(3, 4e-3, 'Wind', bbox=dict(boxstyle="square",ec='k',fc='w'))
    axis.text(0.13, 1e2, 'Stabilizing', bbox=dict(boxstyle="square",ec='k',fc='w'))

In [ ]:
def plot_forcing_regime(hist, xi, yi, axis=None):
    """Plot diagram of forcing regime
    
    """
    if axis is None:
        axis = plt.gca()
    # range of power
    xpr = [-1, 1]
    ypr = [-3, 3]
    # range
    xlims = [10**i for i in xpr]
    ylims = [10**i for i in ypr]
    # background following Fig. 3 of Belcher et al., 2012
    nx = 500
    ny = 500
    xx = np.logspace(xpr[0], xpr[1], nx)
    yy = np.logspace(ypr[0], ypr[1], ny)
    zz1 = np.zeros([nx, ny])
    zz2 = np.zeros([nx, ny])
    zz3 = np.zeros([nx, ny])
    for i in np.arange(nx):
        for j in np.arange(ny):
            zz1[i,j] = 2*(1-np.exp(-0.5*xx[i]))
            zz2[i,j] = 0.22*xx[i]**(-2)
            zz3[i,j] = 0.3*xx[i]**(-2)*yy[j]
    zz = zz1 + zz2 + zz3

    rz_ST = zz1/zz
    rz_LT = zz2/zz
    rz_CT = zz3/zz
    fr = np.ones(zz.shape) * 7
    print(rz_LT.shape)
    print(fr.shape)
    cfrac = 0.25
    fr[(rz_LT<cfrac) & (rz_CT<cfrac)] = 1
    fr[(rz_ST<cfrac) & (rz_CT<cfrac)] = 2
    fr[(rz_ST<cfrac) & (rz_LT<cfrac)] = 3
    fr[(rz_ST>=cfrac) & (rz_LT>=cfrac) & (rz_CT<cfrac)] = 4
    fr[(rz_ST>=cfrac) & (rz_CT>=cfrac) & (rz_LT<cfrac)] = 5
    fr[(rz_LT>=cfrac) & (rz_CT>=cfrac) & (rz_ST<cfrac)] = 6

    color_list = ['firebrick','forestgreen','royalblue','gold','orchid','turquoise','w']
    cb_ticks = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5]
    cmap, norm = from_levels_and_colors(cb_ticks, color_list)
    axis.contourf(xx, yy, np.transpose(fr), cmap=cmap, norm=norm)
    axis.contour(xx, yy, np.transpose(fr), colors='darkgray')
    axis.set_xlim(xlims)
    axis.set_ylim(ylims)
    axis.set_xscale('log')
    axis.set_yscale('log')
    axis.set_xlabel('La$_t$')
    axis.set_ylabel('$h/L_L$')
    axis.set_aspect(aspect=1/3)
    axis.text(0.11, 4e-3, 'Langmuir', bbox=dict(boxstyle="square",ec='k',fc='w'))
    axis.text(3, 4e-3, 'Wind', bbox=dict(boxstyle="square",ec='k',fc='w'))
    axis.text(0.13, 1e2, 'Convection', bbox=dict(boxstyle="square",ec='k',fc='w'))

    # plot bi-dimensional histogram
    plot_dist_4p(hist, xi, yi, axis=axis, colors='white', linestyles='-')

In [ ]:
def add_scatter(la, hLL, axis=None, **kwargs):
    if axis is None:
        axis = plt.gca()
    # range of power
    xpr = [-1, 1]
    ypr = [-3, 3]
    # range
    xlims = [10**i for i in xpr]
    ylims = [10**i for i in ypr]
    xx = np.where(la<xlims[0], xlims[0], la)
    xx = np.where(xx>xlims[1], xlims[1], xx)
    yy = np.where(hLL<ylims[0], ylims[0], hLL)
    yy = np.where(yy>ylims[1], ylims[1], yy)
    # yy = vr12_hLL
#     axis.scatter(xx, yy, s=25, clip_on=False, zorder=10, facecolors='none', linewidths=1.5, **kwargs)
    axis.scatter(xx, yy, s=12, clip_on=False, zorder=10, **kwargs)
#     axis.scatter(xx, yy, s=25, marker='+', clip_on=False, zorder=10, **kwargs)

In [ ]:
def add_range(xrange, yrange, axis=None, **kwargs):
    if axis is None:
        axis = plt.gca()
    # range of power
    xpr = [-1, 1]
    ypr = [-3, 3]
    # range
    xlims = [10**i for i in xpr]
    ylims = [10**i for i in ypr]
    xrange = np.maximum(xlims[0], xrange)
    xrange = np.minimum(xlims[1], xrange)
    yrange = np.maximum(ylims[0], yrange)
    yrange = np.minimum(ylims[1], yrange)
    axis.plot(xrange, yrange, '--', linewidth=2.5, clip_on=False, zorder=9, **kwargs)
    axis.plot(xrange[0], yrange[0], '+', clip_on=False, zorder=9, **kwargs)
    axis.plot(xrange[1], yrange[1], '+', clip_on=False, zorder=9, **kwargs)

In [ ]:
# Figure 1: regime diagram following Belcher et al., 2012

# plot figure a
fig = plt.figure()
fig.set_size_inches(4.5, 4)

# plot only the background of regime diagram
plot_regime_diagram_BG12_background()

# reduce margin
plt.tight_layout()

# save figure
figname = fig_root+'/fig_regime_diagram_'+forcing_reg_type+'_noPDF.png'
plt.savefig(figname, dpi = 300)

# --------

# plot figure b
fig = plt.figure()
fig.set_size_inches(4.5, 4)

# plot regime diagram with pdf overlaid
plot_regime_diagram_BG12(hist, xi, yi)

# reduce margin
plt.tight_layout()

# save figure
figname = fig_root+'/fig_regime_diagram_'+forcing_reg_type+'.png'
plt.savefig(figname, dpi = 300)

# --------

# plot figure c
fig, axarr = plt.subplots(1, 2)
fig.set_size_inches(9, 4)

# plot regime diagram with pdf overlaid
plot_regime_diagram_BG12(hist, xi, yi, axis=axarr[0])
axarr[0].text(5, 2e2, '(a)', fontsize=18, color='black')

# plot bi-dimensional histogram
plot_regime_diagram_heating(hist2, xi2, yi2, axis=axarr[1])
axarr[1].text(5, 2e2, '(b)', fontsize=18, color='black')

# y-shift of legend
yshift = -0.1

# Li and Fox-Kemper, 2017
add_scatter(lf17_la, lf17_hLL, color=c_lf17, axis=axarr[0])
axarr[1].text(0.75, 0.35+yshift, 'LF17', transform=axarr[1].transAxes, color=c_lf17, fontweight='bold')

# McWilliams et al., 1997
add_scatter(msm97_la, msm97_hLL, color=c_msm97, axis=axarr[0])
axarr[1].text(0.75, 0.7+yshift, 'MSM97', transform=axarr[1].transAxes, color=c_msm97, fontweight='bold')

# Li et al., 2005
add_range(l05_la_range_s1, [l05_hLL_s1, l05_hLL_s1], color=c_l05, axis=axarr[0])
add_range([l05_la_s2, l05_la_s2], l05_hLL_range_s2, color=c_l05, axis=axarr[0])
add_scatter(l05_la_s3, l05_hLL_s3, color=c_l05, axis=axarr[1])
axarr[1].text(0.75, 0.65+yshift, 'L05', transform=axarr[1].transAxes, color=c_l05, fontweight='bold')

# Harcourt and D'Asaro, 2008
add_scatter(hd08_la_s1, hd08_hLL_s1, color=c_hd08, axis=axarr[0])
add_scatter(hd08_la_s2, hd08_hLL_s2, color=c_hd08, axis=axarr[0])
add_scatter(hd08_la_s3, hd08_hLL_s3, color=c_hd08, axis=axarr[0])
add_scatter(hd08_la_s4, hd08_hLL_s4, color=c_hd08, axis=axarr[0])
axarr[1].text(0.75, 0.6+yshift, 'HD08', transform=axarr[1].transAxes, color=c_hd08, fontweight='bold')

# Grant and Belcher, 2009
add_scatter(gb09_la, gb09_hLL, color=c_gb09, axis=axarr[0])
axarr[1].text(0.75, 0.55+yshift, 'GB09', transform=axarr[1].transAxes, color=c_gb09, fontweight='bold')

# Van Roekel et al., 2012
add_scatter(vr12_la, vr12_hLL, color=c_vr12, axis=axarr[0])
axarr[1].text(0.75, 0.5+yshift, 'VR12', transform=axarr[1].transAxes, color=c_vr12, fontweight='bold')

# Pearson et al., 2015
add_scatter(p15_la_s1, p15_hLL_s1, color=c_p15, axis=axarr[1]) # heating
add_scatter(p15_la_s2, p15_hLL_s2, color=c_p15, axis=axarr[1]) # heating
axarr[1].text(0.75, 0.45+yshift, 'P15', transform=axarr[1].transAxes, color=c_p15, fontweight='bold')

# Reichl et al., 2016
add_range(r16_la_range, [r16_hLL, r16_hLL], color=c_r16, axis=axarr[0])
axarr[1].text(0.75, 0.4+yshift, 'R16', transform=axarr[1].transAxes, color=c_r16, fontweight='bold')

# reduce margin
# plt.tight_layout()

# save figure
figname = fig_root+'/fig_regime_diagram_'+forcing_reg_type+'_sct.png'
plt.savefig(figname, dpi = 300)

In [ ]:
# Figure 2: forcing regime

# plot figure
fig = plt.figure()
fig.set_size_inches(4.5, 4)

plot_forcing_regime(hist, xi, yi)

# reduce margin
plt.tight_layout()

# save figure
figname = fig_root+'/fig_diag_forcing_regime_'+forcing_reg_type+'.png'
plt.savefig(figname, dpi = 300)

In [ ]:
# Figure 3: Fig. 1 and Fig. 2 together

# plot figure
fig, axarr = plt.subplots(1, 2)
fig.set_size_inches(9, 4)
plot_regime_diagram_BG12(hist, xi, yi, axis=axarr[0])
plot_forcing_regime(hist, xi, yi, axis=axarr[1])

# # reduce margin
# plt.tight_layout()

# save figure
figname = fig_root+'/fig_diag_forcing_regime_'+forcing_reg_type+'_all.png'
plt.savefig(figname, dpi = 300)